# 1 start pods on Azure AKS

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-guide-demo`.

In [ ]:
export PJ_ROOT="${HOME}/example-guide-demo"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-guide-demo
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `example-guide-demo`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## start guide-demo service on Azure AKS

In [ ]:
kubectl apply -f controller/guide-demo-service.yaml

In [ ]:
kubectl get services -l app=guide-demo

example)
```
NAME         TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
guide-demo   ClusterIP   10.0.236.157   <none>        3000/TCP   41s
```

## start guide-demo pod on Azure AKS

find the appropriate position to turn the LED on according to the demo space, and define the environment variables derivered by that position.

In [ ]:
export LED_ON_X="1.5"
export LED_ON_Y="0.0"
export NEIGHBOR_RADIUS="0.4"

In [ ]:
envsubst < controller/guide-demo-deployment.yaml | kubectl apply -f -

In [ ]:
kubectl get pods -l app=guide-demo

example)
```
NAME                          READY   STATUS    RESTARTS   AGE
guide-demo-758dd6fc77-hvq6x   1/1     Running   0          19s
guide-demo-758dd6fc77-lh266   1/1     Running   0          19s
guide-demo-758dd6fc77-zcsc2   1/1     Running   0          19s
```